# akademisyenParsing
#### by Dr. Öğr. Üy. Mehmet Selim Yavuz
###### İstanbul Üniversitesi, Müzikoloji Departmanı
27/10/2022
<br>
Bu kod YÖK Akademik veritabanından dışarıya aktarılan akademisyen listelerini csv formatına dönüştürmek içindir.

Lazım olan kütüphaneleri dahil edelim.

In [465]:
import numpy as np
import pandas as pd
import os

Bütün listeleri okuyup içeri alalım.

In [466]:
workingDir = os.getcwd()
targetDir = "/imports/"
path = workingDir + targetDir

dirList = os.listdir(path)
#print(dirList)

akademisyenList = []
for txtFile in dirList:
    #print(txtFile)
    if txtFile.endswith('.txt'):
        with open(path + txtFile) as f:
            for line in f:
                if line.isspace() != True: #print("!!!!BURADA SIPEYS VAR!!!!")
                    akademisyenList.append(line.rstrip())
    #else:
    #    print("!!!! " + txtFile + " TXT DOSYASI DEĞİL!!!!")

print(akademisyenList[0])

103448, ÖĞRETİM GÖREVLİSİ, KAZIM GÜRKAN, SAKARYA ÜNİVERSİTESİ/DEVLET KONSERVATUVARI/ÇALGI EĞİTİMİ BÖLÜMÜ/TÜRK HALK MÜZİĞİ ANABİLİM DALI/, Güzel Sanatlar Temel Alanı, Müzik, Türk Halk müziği ; Türk Halk Müziği (Yorumculuk), kgurkan[at]sakarya.edu.tr, BD15416E4C4254DB


Artık `akademisyenList`de YÖK formatında bilgiler mevcut.

YÖK formatı: akademisyenID, titr, isim-soyisim, üniversite/fakülte/departman/ABD/, temel_alan, bilim_alanı, anahtar_kelime (varsa) ; anahtar_kelime2 (varsa) ; anahtar_kelime3 (varsa), e-posta_adresi, key

Teker teker bakabilmek için liste objelerini listeye çevirmek gerekiyor.

In [467]:
akademisyen = 0
for index, akademisyen in enumerate(akademisyenList):
    akademisyenList[index] = akademisyen.split(", ") # YÖK formatında virgülle ayrılan alanları ayır.
    akademisyenList[index] = akademisyenList[index][:4] # Temel bilim alanı, bilim alanı ve anahtar kelimelere ihtiyacımız yok.
    #print(akademisyenList[index])

Nispeten temiz bir liste elimizde. Artık üniversite kısmını ayırmak gerekiyor.

In [468]:
for index, akademisyen in enumerate(akademisyenList):
    #print(akademisyen)
    splitList = akademisyen[3].split("/")
    #print(splitList)
    cleanList = akademisyen + splitList
    cleanList.pop(3) # eski listeye ihtiyacımız yok
    #print(cleanList)
    cleanList.pop(4) # fakülte bilgisine ihtiyacımız yok
    #print(cleanList)
    if cleanList[-1] == "": # sonda bazen boşluk var
        cleanList.pop(-1)
        #print(cleanList)
    if len(cleanList) == 6:
        cleanList.pop(4)
        #print(cleanList)
    akademisyenList[index] = cleanList
    #print(akademisyen)

Artık temiz bir akademisyen listemiz mevcut. Kalan format: [0] akademisyenID, [1] titr, [2] tam_isim, [3] üniversite, [4] abd

In [469]:
print(akademisyenList[0])

['103448', 'ÖĞRETİM GÖREVLİSİ', 'KAZIM GÜRKAN', 'SAKARYA ÜNİVERSİTESİ', 'TÜRK HALK MÜZİĞİ ANABİLİM DALI']


Şimdi Gephi'nin istediği node formatına sokalım.

In [470]:
for akademisyen in akademisyenList:
    if len(akademisyen) > 5:
        akademisyen.pop(4)
        akademisyen.pop(-1)
        print(akademisyen)
        print(len(akademisyen))

['165762', 'ÖĞRETİM GÖREVLİSİ', 'EMRE SELAHATTİN YUNKUŞ', 'MERSİN ÜNİVERSİTESİ', 'YAYLI ÇALGILAR ANASANAT DALI']
5
['161817', 'ÖĞRETİM GÖREVLİSİ', 'MEHMET MÜNİF AKALIN', 'MERSİN ÜNİVERSİTESİ', 'YAYLI ÇALGILAR ANASANAT DALI']
5


In [471]:
df = pd.DataFrame(akademisyenList, columns=['ID', 'Titr', 'Label', 'Universite', 'ABD'])

In [472]:
df.dropna(inplace=True)

In [473]:
df.head()

,ID,Titr,Label,Universite,ABD
0,103448,ÖĞRETİM GÖREVLİSİ,KAZIM GÜRKAN,SAKARYA ÜNİVERSİTESİ,TÜRK HALK MÜZİĞİ ANABİLİM DALI
1,117411,ÖĞRETİM GÖREVLİSİ,TUNCAY KARDAŞ,SAKARYA ÜNİVERSİTESİ,TÜRK SANAT MÜZİĞİ ANABİLİM DALI
2,103363,ÖĞRETİM GÖREVLİSİ,METİN GÜLSÜN,SAKARYA ÜNİVERSİTESİ,TÜRK SANAT MÜZİĞİ ANABİLİM DALI
3,122112,ÖĞRETİM GÖREVLİSİ,ERHAN BAYRAM,SAKARYA ÜNİVERSİTESİ,TÜRK SANAT MÜZİĞİ ANABİLİM DALI
4,201910,ÖĞRETİM GÖREVLİSİ,KORKUTALP BİLGİN,SAKARYA ÜNİVERSİTESİ,TÜRK SANAT MÜZİĞİ ANABİLİM DALI


In [474]:
df['Polygon'] = 0
df['Type'] = 'Akademisyen'
print(df.dtypes)
df = df.convert_dtypes()
data_types_dict = {'ID': int}
df = df.astype(data_types_dict)
print(df.dtypes)
df.set_index('ID', inplace=True)
sorted_df = df.sort_values(by='ID')
sorted_df.head()

ID            object
Titr          object
Label         object
Universite    object
ABD           object
Polygon        int64
Type          object
dtype: object
ID             int64
Titr          string
Label         string
Universite    string
ABD           string
Polygon        Int64
Type          string
dtype: object


,Titr,Label,Universite,ABD,Polygon,Type
ID,,,,,,
1398,PROFESÖR,CEMAL YURGA,İNÖNÜ ÜNİVERSİTESİ,MÜZİK EĞİTİMİ ANABİLİM DALI,0,Akademisyen
1914,PROFESÖR,GÖKAY YILDIZ,BURDUR MEHMET AKİF ERSOY ÜNİVERSİTESİ,MÜZİK EĞİTİMİ ANASANAT DALI,0,Akademisyen
1914,PROFESÖR,GÖKAY YILDIZ,BURDUR MEHMET AKİF ERSOY ÜNİVERSİTESİ,MÜZİK EĞİTİMİ ANASANAT DALI,0,Akademisyen
1914,PROFESÖR,GÖKAY YILDIZ,BURDUR MEHMET AKİF ERSOY ÜNİVERSİTESİ,MÜZİK EĞİTİMİ ANASANAT DALI,0,Akademisyen
2320,PROFESÖR,GÜLÇİN YAHYA KAÇAR,ANKARA HACI BAYRAM VELİ ÜNİVERSİTESİ,ÇALGI EĞİTİMİ ANABİLİM DALI,0,Akademisyen


In [475]:
sorted_df.drop_duplicates(subset=None, keep='first', inplace=True)
sorted_df.head()

,Titr,Label,Universite,ABD,Polygon,Type
ID,,,,,,
1398,PROFESÖR,CEMAL YURGA,İNÖNÜ ÜNİVERSİTESİ,MÜZİK EĞİTİMİ ANABİLİM DALI,0,Akademisyen
1914,PROFESÖR,GÖKAY YILDIZ,BURDUR MEHMET AKİF ERSOY ÜNİVERSİTESİ,MÜZİK EĞİTİMİ ANASANAT DALI,0,Akademisyen
2320,PROFESÖR,GÜLÇİN YAHYA KAÇAR,ANKARA HACI BAYRAM VELİ ÜNİVERSİTESİ,ÇALGI EĞİTİMİ ANABİLİM DALI,0,Akademisyen
2587,PROFESÖR,MUSTAFA HİLMİ BULUT,SİVAS CUMHURİYET ÜNİVERSİTESİ,MÜZİK EĞİTİMİ ANASANAT DALI,0,Akademisyen
3015,PROFESÖR,MELİK ERTUĞRUL BAYRAKTARKATAL,BAŞKENT ÜNİVERSİTESİ,BESTECİLİK VE MÜZİK TEORİSİ PR.,0,Akademisyen


In [476]:
print(len(sorted_df.index))

1377


In [477]:
sorted_df.to_csv('nodes.csv', columns=['Type', 'Label', 'Titr', 'Universite', 'ABD', 'Polygon'], encoding="utf-16")

In [478]:
df = sorted_df

In [479]:
uniDF = df.groupby(['Universite', 'ABD']).size().reset_index(name='Sayı')
uniDF.drop('Sayı', axis=1, inplace=True)
uniDF.head()

,Universite,ABD
0,ADIYAMAN ÜNİVERSİTESİ,MÜZİKOLOJİ ANABİLİM DALI
1,AFYON KOCATEPE ÜNİVERSİTESİ,KORO ANASANAT DALI
2,AFYON KOCATEPE ÜNİVERSİTESİ,OPERA ANASANAT DALI
3,AFYON KOCATEPE ÜNİVERSİTESİ,PİYANO ANASANAT DALI
4,AFYON KOCATEPE ÜNİVERSİTESİ,TEMEL BİLİMLER ANASANAT DALI


In [480]:
uniDF_rev = uniDF[['ABD','Universite']].reset_index()
uniDF_rev.drop('index', axis=1, inplace=True)
uniDF.tail()

,Universite,ABD
353,İSTANBUL ÜNİVERSİTESİ,YAYLI ÇALGILAR ANASANAT DALI
354,İSTANBUL ÜNİVERSİTESİ,ÜFLEMELİ VE VURMALI ÇALGILAR ANASANAT DALI
355,İZMİR DEMOKRASİ ÜNİVERSİTESİ,MÜZİKOLOJİ ANABİLİM DALI
356,İZMİR KATİP ÇELEBİ ÜNİVERSİTESİ,MÜZİK ANASANAT DALI
357,ŞIRNAK ÜNİVERSİTESİ,MÜZİK ANASANAT DALI


In [481]:
uniDF.rename(columns={'ABD': 'Target', 'Universite':'Source'}, inplace=True)
uniDF['Polygon'] = int(3)
uniDF.head()

,Source,Target,Polygon
0,ADIYAMAN ÜNİVERSİTESİ,MÜZİKOLOJİ ANABİLİM DALI,3
1,AFYON KOCATEPE ÜNİVERSİTESİ,KORO ANASANAT DALI,3
2,AFYON KOCATEPE ÜNİVERSİTESİ,OPERA ANASANAT DALI,3
3,AFYON KOCATEPE ÜNİVERSİTESİ,PİYANO ANASANAT DALI,3
4,AFYON KOCATEPE ÜNİVERSİTESİ,TEMEL BİLİMLER ANASANAT DALI,3


In [482]:
uniDF_rev.rename(columns={'Universite': 'Target', 'ABD':'Source'}, inplace=True)
uniDF_rev['Polygon'] = int(5)
uniDF_rev.tail()

,Source,Target,Polygon
353,YAYLI ÇALGILAR ANASANAT DALI,İSTANBUL ÜNİVERSİTESİ,5
354,ÜFLEMELİ VE VURMALI ÇALGILAR ANASANAT DALI,İSTANBUL ÜNİVERSİTESİ,5
355,MÜZİKOLOJİ ANABİLİM DALI,İZMİR DEMOKRASİ ÜNİVERSİTESİ,5
356,MÜZİK ANASANAT DALI,İZMİR KATİP ÇELEBİ ÜNİVERSİTESİ,5
357,MÜZİK ANASANAT DALI,ŞIRNAK ÜNİVERSİTESİ,5


In [483]:
totalDF = pd.concat([uniDF,uniDF_rev], axis=0).reset_index()
totalDF.drop('index', axis=1, inplace=True)
totalDF.tail()

,Source,Target,Polygon
711,YAYLI ÇALGILAR ANASANAT DALI,İSTANBUL ÜNİVERSİTESİ,5
712,ÜFLEMELİ VE VURMALI ÇALGILAR ANASANAT DALI,İSTANBUL ÜNİVERSİTESİ,5
713,MÜZİKOLOJİ ANABİLİM DALI,İZMİR DEMOKRASİ ÜNİVERSİTESİ,5
714,MÜZİK ANASANAT DALI,İZMİR KATİP ÇELEBİ ÜNİVERSİTESİ,5
715,MÜZİK ANASANAT DALI,ŞIRNAK ÜNİVERSİTESİ,5


In [484]:
print(len(uniDF.index))
print(len(totalDF.index))

358
716


In [485]:
totalDF.to_csv('uni.csv', encoding="utf-16")

In [486]:
totalDF.reset_index(inplace=True)
totalDF.head()

,index,Source,Target,Polygon
0,0,ADIYAMAN ÜNİVERSİTESİ,MÜZİKOLOJİ ANABİLİM DALI,3
1,1,AFYON KOCATEPE ÜNİVERSİTESİ,KORO ANASANAT DALI,3
2,2,AFYON KOCATEPE ÜNİVERSİTESİ,OPERA ANASANAT DALI,3
3,3,AFYON KOCATEPE ÜNİVERSİTESİ,PİYANO ANASANAT DALI,3
4,4,AFYON KOCATEPE ÜNİVERSİTESİ,TEMEL BİLİMLER ANASANAT DALI,3


In [487]:
totalDF['index'] = totalDF['index']+1000000
totalDF.rename(columns={'index':'ID', 'Source': 'Label'}, inplace=True)
totalDF.head()

ID          int64
Label      string
Target     string
Polygon     int64
dtype: object

In [488]:
totalDF.set_index('ID', inplace=True)
totalDF.head()

,Label,Target,Polygon
ID,,,
1000000,ADIYAMAN ÜNİVERSİTESİ,MÜZİKOLOJİ ANABİLİM DALI,3
1000001,AFYON KOCATEPE ÜNİVERSİTESİ,KORO ANASANAT DALI,3
1000002,AFYON KOCATEPE ÜNİVERSİTESİ,OPERA ANASANAT DALI,3
1000003,AFYON KOCATEPE ÜNİVERSİTESİ,PİYANO ANASANAT DALI,3
1000004,AFYON KOCATEPE ÜNİVERSİTESİ,TEMEL BİLİMLER ANASANAT DALI,3


In [490]:
totalDF.to_csv('uniNode.csv', encoding="utf-16")

### Change log
2022-10-27 created file and wrote the data prep function